In [25]:
from lightning import LitCNN
from utils.transforms import get_transform
from datasets import BeamDataset, collate_fn

from torch.utils import data as torch_data
from pytorch_lightning import Trainer
import math
import copy

In [26]:
# ssh -N -f -L localhost:16006:localhost:6006 kamranzolfonoon@35.203.109.50
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ --port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 31242), started 4 days, 20:34:59 ago. (Use '!kill 31242' to kill it.)

In [27]:
dataset = BeamDataset("/home/kamranzolfonoon/dev/eagle-images/beam_large_batch", "/home/kamranzolfonoon/dev/eagle-images/beam_large_batch/annotations/annotations.json", get_transform(train=True))

train_dataset = torch_data.Subset(dataset, range(0, math.floor(len(dataset) * 0.8)))
test_dataset = torch_data.Subset(dataset, range(math.floor(len(dataset) * 0.8), len(dataset)))

# Janky code to remove training transforms from test dataset
test_dataset.dataset = copy.deepcopy(test_dataset.dataset)
test_dataset.dataset.transforms = get_transform(train=False)

train_loader = torch_data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=1, collate_fn=collate_fn)
test_loader = torch_data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=1, collate_fn=collate_fn)

In [28]:
num_classes = 4

model_name = "fasterrcnn_resnet50_fpn_v2"

model_params = {
    "num_classes": num_classes,
    "pretrained": True,
    "frozen": False
}

optimizer_name = "SGD"

optimizer_params = {
    "lr": 0.005,
    "momentum": 0.9,
    "weight_decay": 0.0005
}

scheduler_name = "StepLR"

scheduler_params = {
    "step_size": 5,
    "gamma": 0.1
}

model = LitCNN(model_name, model_params, optimizer_name, optimizer_params, scheduler_name, scheduler_params)

In [29]:
trainer = Trainer(accelerator='gpu', devices=1, max_epochs=45)
trainer.fit(model, train_loader, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                 | Params
--------------------------------------------------------
0 | model          | FasterRCNN           | 41.3 M
1 | validation_map | MeanAveragePrecision | 0     
2 | test_map       | MeanAveragePrecision | 0     
--------------------------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.258   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/148 [00:00<?, ?it/s]                           

/opt/conda/envs/eagles/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 16: 100%|██████████| 148/148 [02:38<00:00,  1.07s/it, loss=0.168, v_num=21]

In [ ]:
from utils.visualization import visualize_image

# If checkpoint is saved, load it:
# model = LitCNN.load_from_checkpoint('/home/kamranzolfonoon/dev/eaglescnn/lightning_logs/version_4/checkpoints/epoch=14-step=1770.ckpt')
# good example: 12, 59
# tricky: 5, 80, 89, 31, 57
img, target = test_dataset[85]
visualize_image(model,img,target, "prediction.png", "labels.png", threshold=0.75)


In [24]:
# test on full dataset
# model = LitCNN.load_from_checkpoint('/home/kamranzolfonoon/dev/eaglescnn/lightning_logs/version_4/checkpoints/epoch=14-step=1770.ckpt')
full_dataset = BeamDataset("/home/kamranzolfonoon/dev/eagle-images/beam_large_batch", "/home/kamranzolfonoon/dev/eagle-images/beam_large_batch/annotations/annotations.json", get_transform(train=False))
full_test_loader = torch_data.DataLoader(full_dataset, batch_size=4, shuffle=False, num_workers=1, collate_fn=collate_fn)
trainer.test(model, full_test_loader)

588